In [ ]:
# ChatGPT4o 
# prompt:   How to Finetune Stable difdusion with LoRA

In [ ]:
# Step 1: Install Dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate safetensors
!pip install bitsandbytes peft datasets omegaconf xformers rembg gradio

In [ ]:
# Step 2: Download Base Stable Diffusion Model
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe.save_pretrained("./sd_model")

In [ ]:
import os
import json

dataset_path = "./sticker_dataset"
os.makedirs(dataset_path, exist_ok=True)

In [ ]:
# Step 4: Clone and Setup QLoRA Training Scripts
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts
!pip install -r requirements.txt

In [ ]:
# Step 5: Fine-Tune with QLoRA
# Edit the --output_dir and filenames as needed
!accelerate launch train_network.py \
  --pretrained_model_name_or_path="../sd_model" \
  --train_data_dir="../sticker_dataset" \
  --output_dir="../qlora_fine_tuned" \
  --network_module=lycoris.kohya \
  --network_dim=16 \
  --resolution=512 \
  --train_batch_size=2 \
  --learning_rate=1e-4 \
  --max_train_steps=1000 \
  --use_8bit_adam \
  --use_4bit

In [ ]:
# Step 6: Generate Sticker Images with Fine-Tuned Model
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipe.load_lora_weights("../qlora_fine_tuned/sticker_style_qlora.safetensors")

prompt = "sticker-style anime dog, glossy, colorful, outlined"
image = pipe(prompt).images[0]
image.save("generated_sticker.png")
image.show()